In [ ]:
import numpy as np
from utils.dataloader import *
from utils.coordinate_calibration import PointCloudRegistration
from utils.meshgrid import Meshgrid
from utils.icp import IterativeClosestPoint, Matching

In [ ]:
RIGID_BODY_DATA_A = f'./pa345_data/Problem3-BodyA.txt'
RIGID_BODY_DATA_B = f'./pa345_data/Problem3-BodyB.txt'

SURFACE_DATA = f'./pa345_data/Problem3MeshFile.sur'
# SURFACE_DATA = f'./pa345_data/test.sur'

SAMPLE_DATA = f'./pa345_data/PA3-B-Debug-SampleReadingsTest.txt'

rigidbody_dl_A = RigidBodyDataloader.read_file(RIGID_BODY_DATA_A)
rigidbody_dl_B = RigidBodyDataloader.read_file(RIGID_BODY_DATA_B)

# print("rigid body")
# print(rigidbody_dl_A.markers)
# print(rigidbody_dl_A.tip)

surface_dl = Surfaceloader.read_file(SURFACE_DATA)
# print("surface")
# print(surface_dl.N_vertices)
# print(surface_dl.N_triangles)
# print(surface_dl.vertices.shape)
# print(surface_dl.triangles.shape)

sample_dl = SampleReadingsDataloader.read_file(SAMPLE_DATA, delimiter=',', N_A=rigidbody_dl_A.N_markers, N_B=rigidbody_dl_B.N_markers)
# print("sample A", sample_dl.body_A)
# print("sample B", sample_dl.body_B)
# print("dummy", sample_dl.dummy)

### Get F<sub>A, k</sub> and F<sub>B, k</sub>

In [ ]:
# TODO get F_A
rigidbody_dl_A_markers = rigidbody_dl_A.markers.reshape(1, -1, 3) # markers of body A in body A coordinates

sample_dl_A = sample_dl.body_A # samples of body A markers in tracker coordinates
num_samples = sample_dl.N_samps

# perform registration for each frame
reg_F_A = PointCloudRegistration()
F_A = []
for i in range(num_samples):
    sample_dl_A_i = sample_dl_A[i].reshape(1, -1, 3)
    F_A_i, err = reg_F_A.register(rigidbody_dl_A_markers, sample_dl_A_i)
    F_A.append(F_A_i)

F_A = np.array(F_A)

# TODO get F_B
ridigbody_dl_B_markers = rigidbody_dl_B.markers.reshape(1, -1, 3) # markers of body B in body B coordinates
sample_dl_B = sample_dl.body_B # samples of body B markers in tracker coordinates

# perform registration for each frame
reg_F_B = PointCloudRegistration()
F_B = []
for i in range(num_samples):
    sample_dl_B_i = sample_dl_B[i].reshape(1, -1, 3)

    F_B_i, err = reg_F_B.register(ridigbody_dl_B_markers, sample_dl_B_i)
    F_B.append(F_B_i)

F_B = np.array(F_B)

### Get d<sub>k

In [ ]:
A_tip = rigidbody_dl_A.tip
A_tip = np.append(A_tip, 1) # add 1 for homogenous coordinates
d_k = []

for i in range(num_samples):
    F_A_i = F_A[i] # get F_A for frame i
    F_B_i_inv = np.linalg.inv(F_B[i]) # get F_B inverse for frame i
    # print(A_tip.shape)
    # print(F_A_i.shape)
    # print(F_B_i_inv.shape)

    d_k.append(F_B_i_inv @ F_A_i @ A_tip) # d_k = F_B^-1 * F_A * A_tip

d_k = np.array(d_k)[:,:3]
print(d_k.shape)
print(d_k)

In [ ]:
icp = IterativeClosestPoint(Matching.BATCHED_LINEAR)

# print(surface_dl.triangles)
# print(surface_dl.vertices)
print(surface_dl.triangles.shape)


mesh = Meshgrid(surface_dl.vertices, surface_dl.triangles)


# print(surface_dl.vertices[:20])
# print(mesh.trangles[-2:])
pt, dist = icp.match(d_k, mesh)

print(np.around(pt, 2))
print(dist)